In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Define the path for train and test images
train_dir = 'path/to/train/directory'
test_dir = 'path/to/test/directory'

# Create train and validation dataset from the train directory with a batch size of 32
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical'
)

# Create a CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(9, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model for ~20 epochs
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

# Plot the training and validation accuracy and loss
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# Check for model overfit or underfit
if history.history['val_accuracy'][-1] < 0.8:
    print("Model is underfitting")
elif history.history['val_accuracy'][-1] > 0.9:
    print("Model is overfitting")

# Choose an appropriate data augmentation strategy to resolve underfitting/overfitting
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a new train generator with data augmentation
train_generator_augmented = datagen.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical'
)

# Train the model on the augmented data for ~20 epochs
history_augmented = model.fit(
    train_generator_augmented,
    epochs=20,
    validation_data=validation_generator
)

# Plot the training and validation accuracy and loss on the augmented data
plt.plot(history_augmented.history['accuracy'], label='Training Accuracy (Augmented)')
plt.plot(history_augmented.history['val_accuracy'], label='Validation Accuracy (Augmented)')
plt.plot(history_augmented.history['loss'], label='Training Loss (Augmented)')
plt.plot(history_augmented.history['val_loss'], label='Validation Loss (Augmented)')
plt.legend()
plt.show()

# Examine the current class distribution in the training dataset
class_distribution = train_generator.classes
print("Class distribution:", class_distribution)

# Rectify class imbalances present in the training dataset with Augmentor library
from augmentor import Augmentor

augmentor = Augmentor()
augmentor.add_operation(augmentor.operations.FlipLeftRight(probability=0.5))
augmentor.add_operation(augmentor.operations.Rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10))
augmentor.add_operation(augmentor.operations.RandomCrop(probability=0.5, percentage_area=0.5))

augmented_train_generator = augmentor.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical'
)

# Train the model on the rectified class imbalance data for ~30 epochs
history_rectified = model.fit(
    augmented_train_generator,
    epochs=30,
    validation_data=validation_generator
)

# Plot the training and validation accuracy and loss on the rect